#Install Selenium in Colab

In [ ]:
!pip install selenium
!pip install requests
!apt-get update
!sudo apt-get install -y chromium-browser
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 

In [ ]:
import sys, os, time, requests
import urllib.request

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
#wd = webdriver.Chrome('chromedriver', options=chrome_options)

In [ ]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(sleep_between_interactions)

In [ ]:
def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0

    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CSS_SELECTOR, "img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements(By.CSS_SELECTOR, 'img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element(By.CSS_SELECTOR, ".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)
    
    return image_urls

In [ ]:
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
        f.write(image_content)
        f.close()
        print(f"SUCCESS - saved {url} - as {folder_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path, options=chrome_options) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    counter = 0
    for elem in res:
        persist_image(target_folder, elem, counter)
        counter += 1

DRIVER_PATH = '/usr/lib/chromium-browser/chromedriver'
search_term = 'jealous%20face'
# num of images you can pass it from here  by default it's 10 if you are not passing
# number_images = 10
search_and_download(search_term=search_term, driver_path=DRIVER_PATH, target_path='./', number_images=167)

<ipython-input-88-8528dbcaeca7>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path, options=chrome_options) as wd:


Found: 100 search results. Extracting links from 0:100
Found: 168 image links, done!
SUCCESS - saved https://us.123rf.com/450wm/anutaratanya/anutaratanya2005/anutaratanya200500008/anutaratanya200500008.jpg?ver=6 - as ./jealous%20face
SUCCESS - saved https://media.istockphoto.com/id/1282798991/vector/black-girl-thinks-beautiful-face-question-mark.jpg?s=612x612&w=0&k=20&c=Lb7o54HEF9nKohq4aqcm66XluJKnOUW5Wu24kzgzQDA= - as ./jealous%20face
SUCCESS - saved https://media.istockphoto.com/id/1218223808/photo/close-up-depressed-unhappy-indian-woman-thinking-about-problems.jpg?s=612x612&w=0&k=20&c=79MKPsJ0jyRRwB-utGWw_shaSdGB8vUTEd7ESDLBG_8= - as ./jealous%20face
SUCCESS - saved https://image1.masterfile.com/getImage/Njk1LTA1NzY1NzA0ZW4uMDAwMDAwMDA=AFcWww/695-05765704en_Masterfile.jpg - as ./jealous%20face
SUCCESS - saved https://media.istockphoto.com/id/471401024/vector/jealous-emoticon.jpg?s=612x612&w=0&k=20&c=wDnL6n-qR8LNlxTpuv3G_qntGQsNLUljQTaEtKQSjcA= - as ./jealous%20face
SUCCESS - saved h

In [ ]:
!zip -r './jealous face.zip' './jealous%20face'

  adding: jealous%20face/ (stored 0%)
  adding: jealous%20face/jpg_69.jpg (deflated 4%)
  adding: jealous%20face/jpg_68.jpg (deflated 13%)
  adding: jealous%20face/jpg_154.jpg (deflated 1%)
  adding: jealous%20face/jpg_4.jpg (deflated 5%)
  adding: jealous%20face/jpg_59.jpg (deflated 30%)
  adding: jealous%20face/jpg_158.jpg (deflated 1%)
  adding: jealous%20face/jpg_27.jpg (deflated 0%)
  adding: jealous%20face/jpg_133.jpg (deflated 0%)
  adding: jealous%20face/jpg_3.jpg (deflated 29%)
  adding: jealous%20face/jpg_122.jpg (deflated 1%)
  adding: jealous%20face/jpg_97.jpg (deflated 0%)
  adding: jealous%20face/jpg_106.jpg (deflated 14%)
  adding: jealous%20face/jpg_132.jpg (deflated 9%)
  adding: jealous%20face/jpg_140.jpg (deflated 0%)
  adding: jealous%20face/jpg_149.jpg (stored 0%)
  adding: jealous%20face/jpg_12.jpg (deflated 4%)
  adding: jealous%20face/jpg_114.jpg (deflated 3%)
  adding: jealous%20face/jpg_65.jpg (deflated 3%)
  adding: jealous%20face/jpg_33.jpg (deflated 1%)
  a

In [ ]:

!cp './negative face.zip' './neg'
!cp './negative emotion.zip' './neg'
!cp './unhappy face.zip' './neg'
!cp './not funny face.zip' './neg'
!cp './невеселое лицо.zip' './neg'
!cp './несчастливое лицо.zip' './neg'
!cp './негативная эмоция.zip' './neg'
!cp './отрицательная эмоция.zip' './neg'
!cp './несчастное лицо.zip' './neg'

In [ ]:
!zip -r './neg.zip' './neg'

  adding: neg (deflated 0%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp './neg.zip' '/content/drive/MyDrive/Хакатон'
!cp './pos.zip' '/content/drive/MyDrive/Хакатон'

In [ ]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


In [ ]:
!sudo mkdir '/content/drive/MyDrive/Хакатон/архив'

In [ ]:
!unzip '/content/drive/MyDrive/Хакатон/архив.zip' -d '/content/drive/MyDrive/Хакатон/архив'

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im370.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3700.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3701.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3702.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3703.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3704.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3705.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3706.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3707.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3708.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im3709.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sad/im371.png  
  inflating: /content/drive/MyDrive/Хакатон/архив/train/sa

In [ ]:
!cp './neg.zip' '/content/drive/MyDrive/Хакатон'